In [11]:
import pandas as pd
import requests

from api.data import (store_database_for_eys_gene,
                      parse_lovd,
                      parse_gnomad,
                      LOVD_PATH,
                      set_lovd_dtypes,
                      set_gnomad_dtypes,
                      request_gnomad_api_data,
                      merge_gnomad_lovd,
                      GNOMAD_PATH,
                      )
from api.data import save_lovd_as_vcf


pd.options.display.max_columns = 0

In [ ]:
store_database_for_eys_gene("lovd", override=False)

In [ ]:
data = parse_lovd(LOVD_PATH + "/lovd_data.txt")

In [ ]:
gnomad_data = request_gnomad_api_data("EYS")

display(gnomad_data)

In [ ]:
store_database_for_eys_gene('gnomad', False)

gnomad_data_2 = parse_gnomad(GNOMAD_PATH +'/gnomad_data.csv')

In [ ]:
display(gnomad_data_2)

In [ ]:
gnomad_data_2.to_csv('C:\\Users\\Kajus\\Desktop\\gnomad_data_downloaded.csv', index=False)
gnomad_data.to_csv('C:\\Users\\Kajus\\Desktop\\gnomad_data_api.csv', index=False)

In [ ]:
len(gnomad_data_2), len(gnomad_data)

print(len(gnomad_data_2) - len(gnomad_data))

In [12]:
gnomad_data

,Popmax,Popmax population,Homozygote Count,Allele Frequency,variant_id
0,0.000016,African/African American,0.0,1.807419e-06,6-63720525-A-G
1,0.000192,East Asian,0.0,6.573844e-06,6-63720525-A-T
2,0.000000,,0.0,0.000000e+00,6-63720525-A-C
3,0.000020,South Asian,0.0,1.045299e-06,6-63720526-T-A
4,0.000000,,0.0,0.000000e+00,6-63720527-G-T
...,...,...,...,...,...
14295,0.000000,,0.0,0.000000e+00,6-65495479-G-T
14296,0.000031,African/African American,0.0,1.446349e-06,6-65495479-G-A
14297,0.000070,Admixed American,0.0,2.629510e-06,6-65495482-A-G
14298,0.000060,South Asian,0.0,3.645085e-06,6-65495484-T-G


In [ ]:
missing_from_api = []

for i in gnomad_data['gnomAD ID']:
    if(i in gnomad_data_2['gnomAD ID'].values):
        continue
    missing_from_api.append(i)

len(missing_from_api)

missing_data = gnomad_data.loc[gnomad_data['gnomAD ID'].isin(missing_from_api)]

missing_data

In [ ]:
missing_data.to_csv('C:\\Users\\Kajus\\Desktop\\gnomad_data_missing.csv', index=False)

In [ ]:
set_lovd_dtypes(data)
set_gnomad_dtypes(gnomad_data)

variants_on_genome = data["Variants_On_Genome"].copy()

lovd_data = pd.merge(data["Variants_On_Transcripts"],
                       variants_on_genome[['id','VariantOnGenome/DNA/hg38']],
                       on='id',
                       how='left')

gnomad_data = gnomad_data.copy()
final_data = merge_gnomad_lovd(lovd_data, gnomad_data)
final_data

In [ ]:
for i in data:
    print(i)
    display(data[i])

In [ ]:
set_lovd_dtypes(data)
for i in data:
    print(i)
    display(data[i].info())

In [ ]:
save_lovd_as_vcf(data["Variants_On_Genome"], "./lovd.vcf")

In [ ]:
from subprocess import Popen

process = Popen("spliceai -I ./lovd.vcf -O ./lovd_output.vcf -R ../tools/spliceai/hg38.fa -A grch38".split())
process.wait()

In [ ]:
from api.tools import get_revel_scores

chromosome = 6
position = 65655758

results = get_revel_scores(chromosome, position)

display(results)